In [1]:
!pip install pulp

Defaulting to user installation because normal site-packages is not writeable


In [2]:
import random
from itertools import permutations
from pulp import LpProblem, LpVariable, LpStatus, LpMinimize, lpSum
from typing import Dict, List, Tuple
from pulp import GLPK_CMD
import pandas as pd
import networkx as nx
from typing import List, Tuple, Dict

def generate_dummy_data(n_pois: int, n_days: int) -> tuple:
    """
    Generate dummy data for the itinerary optimization problem.

    Parameters:
    - n_pois: The number of points of interest (POIs).
    - n_days: The number of days.

    Returns:
    - pois: A list of POI names.
    - days: A list of day identifiers.
    - travel_time: A dictionary with travel times between each pair of POIs.
    """
    pois = [f'poi_{i}' for i in range(n_pois)]
    days = [f'D{day}' for day in range(1, n_days + 1)]
    travel_time = {(poi1, poi2): random.randint(10, 60) for poi1, poi2 in permutations(pois, 2)}
    return pois, days, travel_time

def collect_results_per_day(edges: List[Tuple[str, str, str]]) -> Dict[str, List[str]]:
    """
    Collects results per day as a directed graph and returns the nodes in each graph
    sorted in topological order.

    Parameters:
    edges (List[Tuple[str, str, str]]): A list of tuples representing the edges.
    
    Returns:
    Dict[str, List[str]]: A dictionary where keys are days and values are lists of POIs
    in topological order.
    """
    graphs_per_day = {}
    results_per_day = {}
    
    for edge in edges:
        source, target, day = edge
        if day not in graphs_per_day:
            graphs_per_day[day] = nx.DiGraph()
        
        if not source.startswith('nonce_node_') and source != target:
            graphs_per_day[day].add_edge(source, target)
    
    for day, graph in graphs_per_day.items():
        # Ensure the graph is a DAG before attempting topological sort.
        if nx.is_directed_acyclic_graph(graph):
            # Filter out nodes starting with 'nonce_node' during topological sort
            sorted_nodes = [node for node in nx.topological_sort(graph) if not node.startswith('nonce_node')]
            results_per_day[day] = sorted_nodes
        else:
            results_per_day[day] = []

    return results_per_day


In [4]:
# Initialize data
n_pois = 22  # Number of POIs
n_days = 7  # Number of days
n_nonce_nodes = 1 # Number of non-POI nodes

nonce_nodes = [f"nonce_node_{str(i)}" for i in range(n_nonce_nodes)]
pois, days, travel_time = generate_dummy_data(n_pois, n_days)
##############################################################################################################################

# Initialize the problem
problem = LpProblem("Itinerary_Optimization", LpMinimize)

# Variables
order_vars = {
    (poi1, poi2, day): LpVariable(f"order_{poi1}_{poi2}_{day}", cat="Binary")
    for poi1 in pois + ['nonce_node'] for poi2 in pois + ['nonce_node'] 
    for day in days 
    if not (poi1=='nonce_node' and poi1==poi2)
}
# Objective function: Minimize the total travel time
problem += lpSum(
    travel_time[(poi_from, poi_to)] * order_vars[(poi_from, poi_to, day)]
    for poi_from, poi_to in permutations(pois, 2) for day in days
)

# Constraints

# Each POI visited exactly once across all days
for poi in pois:
    problem += lpSum(
        order_vars[(poi, other_poi, day)] + order_vars[(other_poi, poi, day)]
        for day in days for other_poi in pois + ['nonce_node'] # if poi != other_poi
    ) == 2 + lpSum(order_vars[(poi, poi, day)] for day in days), f"enter_and_exit_{poi}"

for day in days:
    for poi in pois:
        problem += lpSum(
            order_vars[(poi, other_poi, day)]
            for other_poi in pois + ['nonce_node']
        ) == lpSum(
            order_vars[(other_poi, poi, day)]
            for other_poi in pois + ['nonce_node']
            if poi != other_poi
        ), f"same_day_visit_{poi}_{day}"


# Add sequence variables
sequence_vars = {
    (poi, day): LpVariable(f"sequence_{poi}_{day}", 1, n_pois + 1, cat="Integer")
    for poi in pois for day in days
}

# Sequence constraints
# Ensure sequence is strictly increasing for visits between different POIs
for day in days:
    for poi_from in pois:
        for poi_to in pois:
            if poi_from != poi_to:
                problem += sequence_vars[(poi_from, day)] + 1 <= sequence_vars[(poi_to, day)] + n_pois * (1 - order_vars[(poi_from, poi_to, day)]), f"sequence_{poi_from}_{poi_to}_{day}"

# End-of-day constraints using self-visit logic
# After a POI visits itself, it cannot visit another POI
for day in days:
    for poi in pois:
        problem += sequence_vars[(poi, day)] <= n_pois * (1 - order_vars[(poi, poi, day)]) + (n_pois + 1) * order_vars[(poi, poi, day)], f"end_day_{poi}_{day}"

for day in days:
    problem += lpSum(
        order_vars[('nonce_node', poi, day)] 
        for poi in pois) == 1, f"start_nonce_{day}"
    
    problem += lpSum(
        order_vars[(poi, 'nonce_node', day)] 
        for poi in pois) == 0, f"end_nonce_{day}"

# problem
# Solve the problem
status = problem.solve()

# Check the status and print the results
if LpStatus[status] == 'Optimal':
    print("Solution Found:\n")
    results = []
    for var in order_vars:
        if order_vars[var].varValue == 1:
            results.append(var)
            # print(f"{var} = {order_vars[var].varValue}")
else:
    print("No optimal solution found.")

itinerary = collect_results_per_day(results)


data_for_df = [{'POI': poi, 'DAY': day} for day, pois in itinerary.items() for poi in pois]

# Convert the list of dictionaries into a DataFrame
df_itinerary = pd.DataFrame(data_for_df)
df_itinerary

Solution Found:

('poi_0', 'poi_3', 'D1') = 1
('poi_1', 'poi_7', 'D1') = 1
('poi_2', 'nonce_node', 'D1') = 1
('poi_3', 'poi_2', 'D1') = 1
('poi_4', 'nonce_node', 'D3') = 1
('poi_5', 'poi_1', 'D1') = 1
('poi_6', 'nonce_node', 'D2') = 1
('poi_7', 'poi_0', 'D1') = 1
('nonce_node', 'poi_4', 'D3') = 1
('nonce_node', 'poi_5', 'D1') = 1
('nonce_node', 'poi_6', 'D2') = 1
